In [1]:
import numpy as np

In [5]:
data = "hello"
chars = list(set(data))
chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'h': 0, 'o': 1, 'e': 2, 'l': 3}, {0: 'h', 1: 'o', 2: 'e', 3: 'l'})

In [7]:
# RNN configs
embedding_size = 8
hidden_size = 64
output_size = len(chars)

In [10]:
# Matrices
Wxh = np.random.randn(embedding_size, hidden_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(hidden_size, output_size)
by = np.random.randn(1, output_size)